In [1]:
%cd src

/home/romet/projects/ut/automl/src


In [6]:
import pandas as pd
from dataloader import load_data

from mcnemar_test import create_contingency_table, calculate_mcnemar_test

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
train, test = load_data()
true_pred = train.target.values
#true_pred = y >= 0.5

Reading training data...
Train length:  200000
Loading test...
Removing synthetic data from test


  0%|          | 0/200 [00:00<?, ?it/s]

real_samples_indexes 100000
synthetic_samples_indexes 100000
Frequency encoding...


/home/romet/projects/ut/automl/src/dataloader.py:69: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  traintest[col + '_freq'] = traintest[col].map(traintest.groupby(col).size())


Train and test shape: (200000, 402) (200000, 401)
Test length:  200000
Scaling data...
(200000, 400)
(200000, 400)


## LightGBM McNemar test

In [8]:
baseline_lgbm_df = pd.read_csv("../kfolds/LightGBM__folds5__0.8949656737461088.csv")
optimized_lgbm_df = pd.read_csv("../kfolds/LightGBMOptimized__folds5__0.9067238024867795.csv")

baseline_lgbm_pred = baseline_lgbm_df['target'] >= 0.5
optimized_lgbm_pred = optimized_lgbm_df['target'] >= 0.5

In [9]:
cont_lgbm_df = create_contingency_table(true_pred, baseline_lgbm_pred, optimized_lgbm_pred)
cont_lgbm_df

,nr_correct_baseline,nr_incorrect_baseline
nr_correct_optimized,167248,4801
nr_incorrect_optimized,7164,20787


In [10]:
calculate_mcnemar_test(cont_lgbm_df)

statistic=466.28032, p-value=0.00000000000000000000
Different proportions of errors (reject H0)


## XGBoost McNemar test

In [11]:
baseline_xgb_df = pd.read_csv("../kfolds/XGBoost__folds5__0.8644135937992727.csv")
optimized_xgb_df = pd.read_csv("../kfolds/XGBoostOptimized__folds5__0.9026087657521092.csv")

In [12]:
baseline_xgb_pred = baseline_xgb_df['target'] >= 0.5
optimized_xgb_pred = optimized_xgb_df['target'] >= 0.5

In [13]:
cont_xgb_df = create_contingency_table(true_pred, baseline_xgb_pred, optimized_xgb_pred)
cont_xgb_df

,nr_correct_baseline,nr_incorrect_baseline
nr_correct_optimized,164673,11300
nr_incorrect_optimized,8528,15499


In [14]:
calculate_mcnemar_test(cont_xgb_df)

statistic=387.25242, p-value=0.00000000000000000000
Different proportions of errors (reject H0)
